# Enter text below and click 'Detect' to check for toxicity

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer('english')
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words('english'))

# Function to load and prepare the data
def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    
    # Map the 'class' column to 'labels'
    df['labels'] = df['class'].map({0: "Hate Speech Detected", 1: "Offensive language detected", 2: "No hate and offensive speech"})
    
    # Retain a copy of the 'class' column for later filtering
    class_column = df['class'].copy()
    
    # Select relevant columns
    df = df[['tweet', 'labels', 'class']]  # Keep 'class' for filtering later
    
    return df, class_column

# Clean the text data
def clean(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = [word for word in text.split() if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split()]
    text = " ".join(text)
    return text

# Load the data
df, class_column = load_and_prepare_data(r"../data/twitter_data.csv")

# Apply cleaning
df["tweet"] = df["tweet"].apply(clean)

# Drop rows with missing values
df = df.dropna(subset=['tweet', 'labels'])

# Convert the text data to arrays
x = np.array(df["tweet"])
y = np.array(df["class"])  # Use 'class' column for training

# Vectorize the text data
cv = CountVectorizer()
x_cv = cv.fit_transform(x)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_cv, y, test_size=0.33, random_state=42)

# Train the decision tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Define the labels mapping
labels_mapping = {0: "Hate Speech Detected", 1: "Offensive language detected", 2: "No hate and offensive speech"}

# Create the text box widget with increased width
text_box = widgets.Textarea(
    value='',
    placeholder='Type your text here...',
    description='Text:',
    disabled=False,
    layout=widgets.Layout(width='80%')  # Set the width to 80% of the available space
)

# Create the button widget with vertical alignment
button = widgets.Button(
    description='Detect',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to detect',
    icon='check',
    layout=widgets.Layout(margin='20px 30px 30px 60px')  # Adjust the margin to align vertically
)

output = widgets.Output()

def on_button_click(b):
    with output:
        try:
            output.clear_output()
            input_text = text_box.value
            cleaned_text = clean(input_text)
            input_vec = cv.transform([cleaned_text])
            prediction = clf.predict(input_vec)[0]
            result = labels_mapping[prediction]
            print("Detection:", result)
        except Exception as e:
            print(f"Error: {e}")

button.on_click(on_button_click)

# Align the button and text box vertically
vbox = widgets.VBox([text_box, button])

display(vbox, output)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Output()